# HPE ILO Redfish - Prometheus - Grafana Monitoring

## Connect to the Kubernetes environment

In [ ]:
kubectl get nodes
kubectl -n svtprometheus get pods,svc

## 1. Create the namespace

Create a namespace for your deployment. The example below creates the namespace svtprometheus. 

In [ ]:
# Example content of the namespace.yml file - either run the command above or this one to create the namespace
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: svtprometheus
EOF

## 2. Create the ConfigMap

The connection and monitoring parameters are transferred to the HPE ILO Redfish Prometheus connector as a Kubernetes ConfigMap. 

The ConfigMap can be created using th Python script: createILOcredentials.v1.0.py. 

Requirements:
    - System with Python 3 and the following Python packages installed: 
      - Fernet
      - getpass
      - etree

Run the script with the following command: python3 createILOcredentials.v1.0.py
The script will ask for the following information:
    - username               ILO username (requires read only access) 
    - password               ILO password
    - Logfile                Logfile name
    - Connector Port         TCP Port that the connector uses to publish the counters.
    - Filename               name of the yml-file (<name>.yml) and the configmap: <name>-xml
    - Monitoringintervall    Monitorintervall in seconds
    - ILO IP Address         ILO IP address (asks for it until you enter: 0)
    

It will create the Kubernetes yml-file (<name>.yml) that can be used to  create the Configmap <name>-xml 
    

In [ ]:
pwd
ls -l

In [ ]:
# apply the configmaps 
kubectl apply -f iloprom1.yml
kubectl apply -f iloprom2.yml
kubectl apply -f iloprom3.yml
#kubectl apply -f iloprom4.yml

Check that the configmap is created properly

In [ ]:
kubectl -n svtprometheus get configmap

In [ ]:
kubectl -n svtprometheus delete configmap/ilo-synergy-xml

In [ ]:
kubectl -n svtprometheus describe configmap/iloprom2-xml

## 3. Create the ILO Redfish Connector POD and Service with the Config Maps

### 3.1 Infrastructure & SimpliVity Demo

In [ ]:
# apply the configmaps 
kubectl apply -f iloprom1.yml
kubectl -n svtprometheus describe configmap/iloprom1-xml

In [6]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilo1
  namespace: svtprometheus
  labels:
    app: ilo-infra
spec:
  selector:
    matchLabels:
      app: ilo-infra
  template:
    metadata:
      labels:
        app: ilo-infra
    spec:
      containers:
        - name: ilo-infra
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprom1-xml   # the correct name of the configmap needs to be added here. 
EOF

deployment.apps/ilo1 created


In [7]:
kubectl -n svtprometheus get pods
kubectl -n svtprometheus get deployments

NAME                                            READY   STATUS    RESTARTS   AGE
alletra6k-hpe-array-exporter-64f5fcdc5d-hnpgn   1/1     Running   0          17d
dhci-hpe-array-exporter-74bccf7d76-v54t4        1/1     Running   0          17d
grafana-8596f898ff-jxqfn                        1/1     Running   0          17d
ilo1-696f485c69-hsvrz                           1/1     Running   0          6s
ilo2-5955fd6c9b-79xwf                           1/1     Running   0          17d
ilo3-7f944b58ff-8cz4p                           1/1     Running   0          17d
ilo4-7486bfdc7d-gbkmz                           1/1     Running   0          17d
ilo5-54c7d7b549-zg55n                           1/1     Running   0          17d
primera630-hpe-array-exporter-544d64556-nq7b8   1/1     Running   0          17d
primera650-hpe-array-exporter-6dfcd6688-ngn9b   1/1     Running   0          17d
prometheus-75bf698fff-2kpdb                     1/1     Running   0          17d
svtdemo-854bd96c8c-jz95b     

In [4]:
kubectl -n svtprometheus delete deployment/ilo1

deployment.apps "ilo1" deleted


In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: iloredfish
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilo-infra
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

### 3.2 Group 2 - Synergy Systems

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: ilo2
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilo2
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

In [1]:
# apply the configmaps 
kubectl apply -f iloprom2.yml
kubectl -n svtprometheus describe configmap/iloprom2-xml

configmap/iloprom2-xml unchanged
Name:         iloprom2-xml
Namespace:    svtprometheus
Labels:       <none>
Annotations:  <none>

Data
====
iloprometheus.xml:
----
<data> 
  <username>Prometheus</username>
  <user>gAAAAABig6ZJgST83R7v5AFjsOKiYt0bwqBdledJpF3-TnUBR5871eNttG8P-O_eYA55-5NyyZgmmeWRnWy3hPAbUFk18_sWNw==</user>
  <password>gAAAAABig6ZJAZL8wS41b0snZ76VWo17JysUq0-k5VjnrNUCU1y5AAPw5p1MphkFcB_u6z0TgcPbmR9yJcX39Cx0Egwv5Lou8A==</password>
  <monitoringintervall>30</monitoringintervall>
  <logfile>iloprom2.log</logfile>
  <port>9091</port>
  <server> <ILO_ip>10.0.20.66</ILO_ip>,<Rack>H5</Rack>,<Loc>HE01b1</Loc> </server>
  <server> <ILO_ip>10.0.20.69</ILO_ip>,<Rack>H5</Rack>,<Loc>HE01b2</Loc> </server>
  <server> <ILO_ip>10.0.20.72</ILO_ip>,<Rack>H5</Rack>,<Loc>HE01b3</Loc> </server>
  <server> <ILO_ip>10.0.20.75</ILO_ip>,<Rack>H5</Rack>,<Loc>HE01b4</Loc> </server>
  <server> <ILO_ip>10.0.20.78</ILO_ip>,<Rack>H5</Rack>,<Loc>HE01b5</Loc> </server>
  <server> <ILO_ip>10.0.20.81</ILO_i

In [ ]:
kubectl -n svtprometheus describe configmap/iloprom2-xml

In [6]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilo2
  namespace: svtprometheus
  labels:
    app: ilo2
spec:
  selector:
    matchLabels:
      app: ilo2
  template:
    metadata:
      labels:
        app: ilo2
    spec:
      containers:
        - name: ilo2
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprom2-xml   # the correct name of the configmap needs to be added here. 
EOF

deployment.apps/ilo2 created


In [4]:
kubectl -n svtprometheus delete deployment/ilo2

deployment.apps "ilo2" deleted


In [3]:
kubectl -n svtprometheus get deployments
kubectl -n svtprometheus get pods
kubectl -n svtprometheus get svc

NAME                            READY   UP-TO-DATE   AVAILABLE   AGE
alletra6k-hpe-array-exporter    1/1     1            1           149d
dhci-hpe-array-exporter         1/1     1            1           371d
grafana                         1/1     1            1           80d
ilo1                            1/1     1            1           56d
ilo2                            1/1     1            1           56d
ilo3                            1/1     1            1           58d
ilo4                            1/1     1            1           54d
ilo5                            1/1     1            1           56d
primera630-hpe-array-exporter   1/1     1            1           371d
primera650-hpe-array-exporter   1/1     1            1           371d
prometheus                      1/1     1            1           57d
svtdemo                         1/1     1            1           5m38s
svtinfra                        1/1     1            1           404d
NAME                       

In [ ]:
kubectl -n svtprometheus logs pod/ilo2-5955fd6c9b-tsm8g

### 3.3 Group 3

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: ilo3
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilo3
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

In [ ]:
kubectl apply -f iloprom3.yml


In [8]:
kubectl -n svtprometheus describe configmap/iloprom3-xml

Name:         iloprom3-xml
Namespace:    svtprometheus
Labels:       <none>
Annotations:  <none>

Data
====
iloprometheus.key:
----
LVkjkV5j40Fl6U0O7ratU0RDf5W1NmAE0oqoQ2DaE74=
iloprometheus.xml:
----
<data> 
  <username>Prometheus</username>
  <user>gAAAAABig6ZJgST83R7v5AFjsOKiYt0bwqBdledJpF3-TnUBR5871eNttG8P-O_eYA55-5NyyZgmmeWRnWy3hPAbUFk18_sWNw==</user>
  <password>gAAAAABig6ZJAZL8wS41b0snZ76VWo17JysUq0-k5VjnrNUCU1y5AAPw5p1MphkFcB_u6z0TgcPbmR9yJcX39Cx0Egwv5Lou8A==</password>
  <monitoringintervall>30</monitoringintervall>
  <logfile>iloprom2.log</logfile>
  <port>9091</port>
  <server> <ILO_ip>10.1.80.11</ILO_ip>,<Rack>G5</Rack>,<Loc>7</Loc> </server>
  <server> <ILO_ip>10.1.80.12</ILO_ip>,<Rack>G5</Rack>,<Loc>5</Loc> </server>
  <server> <ILO_ip>10.1.80.13</ILO_ip>,<Rack>G5</Rack>,<Loc>3</Loc> </server>
  <server> <ILO_ip>10.1.80.14</ILO_ip>,<Rack>G5</Rack>,<Loc>1</Loc> </server>
  <server> <ILO_ip>10.1.43.30</ILO_ip>,<Rack>F5</Rack>,<Loc>35</Loc> </server>
  <server> <ILO_ip>10.0.

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilo3
  namespace: svtprometheus
  labels:
    app: ilo3
spec:
  selector:
    matchLabels:
      app: ilo3
  template:
    metadata:
      labels:
        app: ilo3
    spec:
      containers:
        - name: ilo3
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprom3-xml   # the correct name of the configmap needs to be added here. 
EOF

In [ ]:
kubectl -n svtprometheus get deployments
kubectl -n svtprometheus get pods
kubectl -n svtprometheus get svc

In [ ]:
kubectl -n svtprometheus delete deployment/ilo3
#kubectl -n svtprometheus delete svc/ilo3

In [ ]:
kubectl -n svtprometheus logs pod/ilo1-696f485c69-hwbzm 

In [ ]:
kubectl -n svtprometheus get configmaps

### 3.4 Group 4

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: ilo4
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilo4
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

In [9]:
kubectl apply -f iloprom4.yml
kubectl -n svtprometheus describe configmap/iloprom4-xml

configmap/iloprom4-xml configured
Name:         iloprom4-xml
Namespace:    svtprometheus
Labels:       <none>
Annotations:  <none>

Data
====
iloprometheus.key:
----
LVkjkV5j40Fl6U0O7ratU0RDf5W1NmAE0oqoQ2DaE74=
iloprometheus.xml:
----
<data> 
  <username>Prometheus</username>
  <user>gAAAAABig6ZJgST83R7v5AFjsOKiYt0bwqBdledJpF3-TnUBR5871eNttG8P-O_eYA55-5NyyZgmmeWRnWy3hPAbUFk18_sWNw==</user>
  <password>gAAAAABig6ZJAZL8wS41b0snZ76VWo17JysUq0-k5VjnrNUCU1y5AAPw5p1MphkFcB_u6z0TgcPbmR9yJcX39Cx0Egwv5Lou8A==</password>
  <monitoringintervall>30</monitoringintervall>
  <logfile>iloprom2.log</logfile>
  <port>9091</port>
  <server> <ILO_ip>10.1.43.11</ILO_ip>,<Rack>i12</Rack>,<Loc>1</Loc> </server>
  <server> <ILO_ip>10.1.43.12</ILO_ip>,<Rack>i12</Rack>,<Loc>12</Loc> </server>
  <server> <ILO_ip>10.1.24.41</ILO_ip>,<Rack>D5</Rack>,<Loc>11</Loc> </server>
  <server> <ILO_ip>10.1.24.42</ILO_ip>,<Rack>D5</Rack>,<Loc>9</Loc> </server>
  <server> <ILO_ip>10.1.24.43</ILO_ip>,<Rack>D5</Rack>,<Loc>7</Lo

In [11]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilo4
  namespace: svtprometheus
  labels:
    app: ilo4
spec:
  selector:
    matchLabels:
      app: ilo4
  template:
    metadata:
      labels:
        app: ilo4
    spec:
      containers:
        - name: ilo4
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprom4-xml   # the correct name of the configmap needs to be added here. 
EOF

deployment.apps/ilo4 created


In [12]:
kubectl -n svtprometheus get deployments
kubectl -n svtprometheus get pods
kubectl -n svtprometheus get svc

NAME                            READY   UP-TO-DATE   AVAILABLE   AGE
alletra6k-hpe-array-exporter    1/1     1            1           94d
dhci-hpe-array-exporter         1/1     1            1           316d
grafana                         1/1     1            1           25d
ilo1                            1/1     1            1           27h
ilo2                            1/1     1            1           26h
ilo3                            1/1     1            1           3d6h
ilo4                            1/1     1            1           5s
ilo5                            1/1     1            1           27h
primera630-hpe-array-exporter   1/1     1            1           317d
primera650-hpe-array-exporter   1/1     1            1           317d
prometheus                      1/1     1            1           2d21h
svtdemo                         1/1     1            1           135d
svtinfra                        1/1     1            1           349d
NAME                       

In [10]:
kubectl -n svtprometheus delete deployment/ilo4

deployment.apps "ilo4" deleted


### 3.5 Group 5

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: ilo5
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilo5
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

In [ ]:
kubectl apply -f iloprom5.yml
kubectl -n svtprometheus describe configmap/iloprom5-xml

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilo5
  namespace: svtprometheus
  labels:
    app: ilo5
spec:
  selector:
    matchLabels:
      app: ilo5
  template:
    metadata:
      labels:
        app: ilo5
    spec:
      containers:
        - name: ilo5
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprom5-xml   # the correct name of the configmap needs to be added here. 
EOF

In [ ]:
kubectl -n svtprometheus delete deployment/ilo5

In [ ]:
kubectl -n svtprometheus get deployments
kubectl -n svtprometheus get pods
kubectl -n svtprometheus get svc

# Delete the Deployment and the Service

In [ ]:
kubectl -n svtprometheus delete deployment/iloredfish

In [ ]:
kubectl -n svtprometheus delete svc/ilosynergy